# Tutorial

Optional: [click here to run this tutorial interactively in the cloud -- (coming soon)](...) on a free service called binder. This allows you to test out *ipcoal* without having to even install it. 

### Getting started
Welcome to the quick guide tutorial for *ipcoal*. This page is intended to introduce major concepts of coalescent simulation, and to 
provide a short overview of several types of analyses that can be performed with *ipcoal*. For a more detailed dive into the specifics please visit the User Guide next. 

*ipcoal* is designed for use within jupyter notebooks to make it easy to *interactively* perform analyses alongside visualization tools that make it easy to verify your results. The Python package *toytree* is installed alongside *ipcoal* and should typically be imported with it, like below, as the two are intended to work hand in hand. 

In [74]:
import ipcoal
import toytree

### The species tree
The species tree is the primary *model* on which *ipcoal* is designed to simulate genealogies and sequences within the multispecies coalescent framework. One of the primary features of *ipcoal* is the ability to feed it a tree which it will then parse to build a demographic model (which is used by the `msprime` coalescent simulator), and which describes when and how different populations (lineages) are able to coalesce with each other. You can think of coalescence on a species tree as several distinct coalescent processes occurring within panmictic populations that are simply connected to each other by the tree structure (See Degnan and Rosenberg 2009 for a nice description: https://www.sciencedirect.com/science/article/pii/S0169534709000846). 

To simulate genealogies and sequences on a tree we need to first define the tree. This can be done by either loading an inferred tree from a newick string or by generating a random tree. For this we will use the tree manipulation and visualization library [toytree](https://toytree.readthedocs.io). In the example cell below I use toytree to generate a random tree with a set number of tips, a total tree height, and a random seed, and store it as a variable named `tree1`. 

We can visualize this tree by calling `.draw()` from the toytree and here I provide the argument `tree_style='p'` to set a style for drawing the figure that will make it look nice for representing 'population trees' (i.e., species trees). This is helpful in that it provides numeric labels on the nodes of the tree, shows tip labels, and provides a scalebar for the height of the tree. 

<div class="alert alert-info">

**Note:** Branch lengths on species trees should be in units of *generations*.

Please see the FAQs for tips on translating branch lengths on empirical trees from absolute time to generations. 

</div>


In [75]:
# generate a random tree with 8 tips and height of 1M generations
tree1 = toytree.rtree.unittree(8, treeheight=1e6, seed=123)

In [76]:
# draw tree showing idx labels
tree1.draw(tree_style='p');

<svg class="toyplot-canvas-Canvas" height="260.0px" id="t249c6138118d4d489a222bdb341b8eea" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r6 r4 r7 r2 r5 r0 r3 r1 idx: 0
name: r6
dist: 500000
support: 100
height: 0 0 idx: 1
name: r4
dist: 500000
support: 100
height: 0 1 idx: 2
name: r7
dist: 500000
support: 100
height: 0 2 idx: 3
name: r2
dist: 500000
support: 100
height: 0 3 idx: 4
name: r5
dist: 250000
support: 100
height: 0 4 idx: 5
name: r0
dist: 250000
support: 100
height: 0 5 idx: 6
name: r3
dist: 500000
support: 100
height: 0 6 idx: 7
name: r1
dist: 750000
support: 100
height: 0 7 idx: 8
name: 8
dist: 250000
support: 100
height: 250000 8 idx: 9
name: 9
dist: 250000
support: 100
height: 500000 9 idx: 10
name: 10
dist: 250000
support: 100
height: 500000 10 idx: 11
name: 11
dist: 250000
support: 100
height: 500000 11 idx: 12
name: 12
dist: 250000
support: 100
height: 750000 12 idx: 13
name: 13
dist: 250000
support: 100
height: 750000 13 idx: 14
name: 14
dist: 250000
support: 100
height: 1000000 14 0 500000 1000000

### The demographic model

*ipcoal* is a an object-oriented library of which the main object that users interact with is called the `Model` class object. To create a Model object you must provide a number of parameter arguments to `ipcoal.Model()` which will return a parameterized Model that you can store as a variable. By convention, I typically name it something with "model" in its name. 

From the Model object you can then call functions to simulate genealogies and sequences, and similarly, the results of simulations can also be accessed directly from the Model object.

Here we will start by setting up a simple Model for simulating coalescent genealogies on the species tree plotted above. Because this is our first model we will keep it simple and apply a single effective population size (Ne) to the entire tree, and leave all other parameter arguments at their default values. We will see further on in this tutorial the effect of varying the size of populations. 

In [90]:
# initialize a model object given a species tree and Ne setting.
model = ipcoal.Model(tree=tree, Ne=1e5)

<div class="alert alert-info">
    
**Note**: Explore using interactivity.

One of the benefits of working interactively in a jupyter notebook is that you can easily explore the functionality of your objects. To see a list of all of the attributes and functions that are accessible from a Model class object simply type `model.<tab>` in a code cell below and place your cursor after the dot and press the tab key (once or twice for more information). Do not write the word `<tab>`. This will raise a pop-up window next to your cursor listing options associated with Model objects. 

</div>

### Simulating genealogies
An important distinction that we highlight in *ipcoal* is whether you are simulating *linked* or *unlinked* data. Unlinked data represents *independent* draws from a distribution, whereas linked data represents *correlated* draws, meaning that the next data point is influenced by the preceding one. 

In the context of a genome we expect that regions located on different chromosomes are independent of each other, whereas sites that are located close together on the same chromosome are not independent. In the context of the coalescent, the correlation among nearby regions of the genome represents that one or more samples shares the same ancestors in both regions. Recombination causes this similarity to decay since it has the effect of causing different genomic regions to trace back to different sampled ancestors.  

There are instances where we may be interested in simulating linked data to study the effect of recombination, or alternatively, we may sometimes wish to simulate unlinked data. Many population genetic and phylogenetic inference tools assume that data are unlinked. One useful application of *ipcoal* is to generate linked and unlinked datasets to explore the effect of linkage on the analysis results.  

#### Simulating (unlinked) genealogies
We will start our simulations by focusing first just on genealogies (we will add in sequence simulations later.) To simulate just genealogies -- the fastest type of simulation in *ipcoal* -- you can use the `sim_trees()` function call. This takes two arguments, `nloci` and `nsites`. In *ipcoal* we always treat loci as being independent of one another. You can think of them as separate chromosomes. The length of each locus is represented by some number of sites. To simulate completely unlinked sites we can request a single site (nsites=1) from multiple loci. 

In [101]:
# simulate unlinked genealogies 
unlinked = ipcoal.Model(tree=tree, Ne=1e5, seed=1234)
unlinked.sim_trees(nloci=10, nsites=1)

<div class="alert alert-info">

**Note**: Why do you need to specify nsites when only simulating genealogies and not sequences?

If the simulation includes recombination (which it does by default) then a single locus extending over more than one site may actually represent multiple coalescent genealogies if a recombination crossover occurred in the history of the sample at that locus. If you want unlinked genealogies then you should simulate trees with nsites=1.


</div>

#### Simulating (linked) genealogies
To better understand the difference between linked and unlinked genealogies let's also produce a set of linked genealogies. To do this we simply need to call `sim_trees()` with nsites set >1, and to use a Model object that includes recombination. Our Model object actually already includes recombination since the default parameter setting is `recomb=1e-9`. Here I write it out just to be more explicit. I will simulate one long locus on which we expect multiple genealogies will be represented. 

In [102]:
# simulate linked genealogies
linked = ipcoal.Model(tree=tree, Ne=1e5, recomb=1e-9, seed=1234)
linked.sim_trees(nloci=1, nsites=100000)

### The results dataframe (.df)

The genealogical results of a simulation call are available from a Model object as a table (a Pandas DataFrame), that can be accessed from its `.df` attribute. Above we created two Model objects, called linked and unlinked, which each has its own table of results. Below I show the first ten trees in each dataset. 

In [103]:
# show unlinked genealogies
unlinked.df.head(10)

,locus,start,end,nbps,nsnps,genealogy
0,0,0,1,1,0,"((r0:859988,(r6:666450,r..."
1,1,0,1,1,0,"((r1:545876,r7:545876):1..."
2,2,0,1,1,0,"(((r7:588489,(r1:328153,..."
3,3,0,1,1,0,"((r0:815025,(r6:519779,r..."
4,4,0,1,1,0,"((r0:932310,(r6:727311,r..."
5,5,0,1,1,0,"((r0:812247,(r6:691486,r..."
6,6,0,1,1,0,"(((r3:586938,r2:586938):..."
7,7,0,1,1,0,"(((r7:703730,(r1:293885,..."
8,8,0,1,1,0,"((r6:714476,r5:714476):6..."
9,9,0,1,1,0,"((r0:950022,(r6:777823,r..."


In [104]:
# show linked genealogies
linked.df.head(10)

,locus,start,end,nbps,nsnps,genealogy
0,0,0,649,649,0,"(r5:1181063,((r6:757263,..."
1,0,649,734,85,0,"((((r1:378703,r4:378703)..."
2,0,734,993,259,0,"((((r1:378703,r4:378703)..."
3,0,993,1089,96,0,"((((r1:378703,r4:378703)..."
4,0,1089,1098,9,0,"((((r1:378703,r4:378703)..."
5,0,1098,1162,64,0,"((((r7:766814,(r3:540236..."
6,0,1162,1997,835,0,"((((r7:766814,(r3:540236..."
7,0,1997,2005,8,0,"((((r3:766814,(r7:759555..."
8,0,2005,2052,47,0,"((((r3:766814,(r7:759555..."
9,0,2052,2215,163,0,"((((r3:766814,(r7:759555..."


So what do these tables show? You can see in the **unlinked** results table that 10 different loci are represented, numbered 0-9 in the "locus" column. Each locus is represented by only a single site, stretching from start=0 to end=1. Each is 1bp in length and contains no SNPs since we have not simulated sequence data yet, only genealogies. Finally, the results of interest are in the final column, genealogy, which contains a newick string. 

Now look at the **linked** results table. In contrast to the previous table we see that now all of the data in on a single locus (locus=0). The first genealogy stretches from position 0 (start=0) to position 5 (end=5) and it is 5bp in length. Following down the table we can see that recombination has broken this locus into many small chunks each represented by a slightly different genealogy. 

Of course it is hard to tell just how different these genealogies are within or across loci from just looking at this table. That is why the next step is to use visualization tools and statistical analyses to compare trees. 

### Drawing trees
See the documentation section on Drawing genealogies for more examples of methods and arguments for drawing and styling species trees, gene trees, or genealogies either individually or together on a canvas. 

Here I will focus on how different parameterizations of the Model objects lead to different distributions of genealogies. Below I use the toytree multitree function `mtree` to load the entire list of trees from each object's `.df` table. Then from each multitree object I call the `draw_tree_grid()` function to draw the first few trees in each list with a set of styling options defined in a dictionary. This is a quick way to visualize the variation among a few simulated trees.  Do not worry if it seems complicated for now, this is just for the purposes of visualizing the genealogies. 

In [105]:
# a dictionary of arguments to style the drawings
kwargs = {
    "ts": "c",
    "tip_labels": True, 
    "shared_axis": True,
    "width": 600, 
    "height": 200,
}

# draw a grid of trees from model 1
toytree.mtree(unlinked.df.genealogy).draw_tree_grid(**kwargs);

# draw a grid of trees from model 2
toytree.mtree(linked.df.genealogy).draw_tree_grid(**kwargs);

<svg class="toyplot-canvas-Canvas" height="200.0px" id="t66068df55b82464181e22497a8b84382" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 200.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r2 r3 r7 r4 r1 r5 r6 r0 r0 r5 r6 r2 r4 r3 r7 r1 r4 r1 r7 r2 r3 r5 r6 r0 r4 r1 r7 r2 r3 r5 r6 r0 r4 r1 r7 r2 r3 r5 r6 r0 0 826588 1653175

<svg class="toyplot-canvas-Canvas" height="200.0px" id="td547ec6a3ace4e878bebabe082160a0e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 200.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r2 r3 r7 r4 r1 r0 r6 r5 r2 r3 r7 r4 r1 r0 r5 r6 r2 r3 r7 r4 r1 r0 r5 r6 r2 r3 r7 r4 r1 r0 r5 r6 r2 r3 r7 r4 r1 r0 r5 r6 0 590532 1181064

**Why do the distributions of unlinked and linked trees above look so different?** Because of the effect of linkage, of course. The unlinked trees in the top row represent the full range of genealogical variation across the genome. Each is an independent draw. The linked trees in the lower row, by contrast, represent one single draw from the distribution of genealogies, followed by a series of linked genealogies that differ from the initial tree by small changes in one or a few ancestral coalescences. You can see that the topology only changes slightly, or sometimes stays the same and only an edge lenth changes, and this cccurs in an ordered way from left to right. The trees are correlated. 

In [106]:
# create two models that differ in Ne
model1 = ipcoal.Model(tree=tree, Ne=1e4)
model2 = ipcoal.Model(tree=tree, Ne=1e6)


### Demographic parameters (Ne)
Now that we understand how linkage works let's explore other parameters of the demographic model, starting with the effective population size, Ne. Below I create two models with different Ne values and simulate 10 independent genealogies on each. 

In [107]:
# simulate n genealogies for each model
model1.sim_trees(10)
model2.sim_trees(10)

Once again, we will use tree drawings to visualize the effect of Ne on the genealogical variation. In the plot below the first row corresponds to trees simulated from model 1, where Ne=1e4, and the second row corresonds to trees simulated from model 2, where Ne=1e6. 

As we should expect, when Ne is small the coalescent events occur more quickly and in this case the genealogies all match the species tree. When Ne is larger we instead see much deeper coalescent times (note the y-axis differences in numbers of generations) and the topology is often different from the species tree since the coalescent times trace back deeper than most species tree divergence events. 

This plot demonstrates nicely the importance of Ne in determing genealogical variation: both datasets were simulated on the same species tree but different Ne. 

In [108]:
# a dictionary of arguments to style the drawings
kwargs = {
    "ts": "c",
    "tip_labels": True, 
    "shared_axis": True,
    "width": 600, 
    "height": 200,
}

# draw a grid of trees from model 1
toytree.mtree(model1.df.genealogy).draw_tree_grid(**kwargs);

# draw a grid of trees from model 2
toytree.mtree(model2.df.genealogy).draw_tree_grid(**kwargs);

<svg class="toyplot-canvas-Canvas" height="200.0px" id="t0a2caaaadad14438ba32fd7c76d34858" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 200.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r4 r1 r7 r2 r3 r5 r6 r0 r4 r1 r7 r2 r3 r5 r6 r0 r4 r1 r7 r2 r3 r5 r6 r0 r4 r1 r7 r2 r3 r5 r6 r0 r4 r1 r7 r2 r3 r5 r6 r0 0 516434 1032869

<svg class="toyplot-canvas-Canvas" height="200.0px" id="t8e14d08f1d5a4482a96739d5e83036ea" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 200.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r6 r1 r2 r0 r3 r5 r4 r7 r2 r4 r3 r6 r7 r1 r5 r0 r2 r7 r0 r4 r1 r6 r5 r3 r5 r6 r4 r1 r0 r2 r3 r7 r7 r4 r1 r2 r0 r5 r6 r3 0 3015616 6031232

### Demographic parameters (admixture)
Now that we understand how linkage works let's explore other parameters of the demographic model, starting with the effective population size, Ne. Below I create two models with different Ne values and simulate 10 independent genealogies on each. 

### Simulating sequences
Sequences are simulated on genealogies to produce a sequence alignment stretching over the length of a locus. Because as we learned above a locus can be represented by multiple distinct genealogies in the presence of recombination, sequence data simulated over a locus of length N may in fact represent sequence data simulated over genealogy A for part of its length, and genealogy B for another part of its length. Thus the concept of simulating linked or unlinked genealogies extends naturally to the simulation of linked or unlinked sequence data. 

We make one additional distinction in the terminology of simulating sequence data which is between simulating *loci* and simulating *SNPs*. Simulation of SNPs is a special case where the simulation is conditioned on returned variable sites, and thus it could potentially run forever waiting for a substitution depending on the parameter settings. Follow along below where we describe this distinction between the two sequence simulation functions `.sim_loci()` and `.sim_snps()`. 

#### Simulating (linked) sequences
The `.sim_loci()` function is used for simulating linked sequences, either with or without recombination.  

In [110]:
# create a Model and simulate a 1Kb locus
model = ipcoal.Model(tree, Ne=1e5, recomb=1e-9)
model.sim_loci(nloci=1, nsites=1000)

In [114]:
# the results table
model.df

,locus,start,end,nbps,nsnps,genealogy
0,0,0,352,352,27,"((r0:837218,(r6:599601,r..."
1,0,352,425,73,2,"((r0:837218,(r6:599601,r..."
2,0,425,488,63,3,"((r0:837218,(r6:599601,r..."
3,0,488,505,17,0,"(((r1:374964,r4:374964):..."
4,0,505,623,118,9,"(((r2:766412,(r3:765757,..."
5,0,623,1000,377,42,"(((r2:766412,(r3:765757,..."


In [115]:
# the results sequence array
model.seqs

array([[[1, 0, 1, ..., 1, 1, 3],
        [1, 0, 1, ..., 1, 1, 3],
        [1, 0, 1, ..., 1, 1, 3],
        ...,
        [2, 0, 1, ..., 1, 1, 3],
        [1, 0, 1, ..., 1, 1, 3],
        [1, 0, 1, ..., 1, 1, 3]]], dtype=uint8)

#### Writing sequence data to file

### Simulating (unlinked) SNPs


### Analysis tools

We plan to expand analysis tools...

#### Inferring gene trees from sequences

A useful applicatin of *ipcoal* is in generating sequence data sets for verifying various types of inference tools, such as phylogenetic inference software. 


and once created, you call functions from this object and access results from it. In other words, once you provide the arguments to set up your model you can use it in many interesting ways. 

aThe ratio of edge lengths (the *height* of a internode) in units of generations, and the effective population size over that edge (the *width* of the internode) defines the probability that samples will coalesce over each edge. 

<div class="alert alert-info">

Shorter edges and larger Ne values increase the probability of  discordance, i.e., the genealogy not matching the species tree. When Ne is very small (e.g., 2) two samples will always coalescence on an edge before the next speciation event (looking backwards in time). When Ne is very large (e.g., 1e8) then two samples may have a very deep coalescent time that occurs farther back  

As coalescent events occur deeper in time they are more likely to 

, since more genealogies will coalesce deeper in the tree. At infinite population sizes the topology is random a random variable 

</div>

#### 3a) Simulate unlinked snps

In [7]:
# simulate N unlinked SNPs (will run until N snps are produced)
model.sim_snps(1000)

In [8]:
model.df

,locus,start,end,nbps,nsnps,genealogy
0,0,0,1,1,1,"(r6:3.33677e+06,((r3:737..."
1,1,0,1,1,1,"((r3:1.02472e+06,r5:1.02..."
2,2,0,1,1,1,"(r0:4.08496e+06,((r7:2.4..."
3,3,0,1,1,1,"((r2:1.05782e+06,r0:1.05..."
4,4,0,1,1,1,"((r2:1.50159e+06,(r6:1.0..."
...,...,...,...,...,...,...
995,995,0,1,1,1,"((r6:941804,r3:941804):1..."
996,996,0,1,1,1,"((r5:2.38812e+06,r7:2.38..."
997,997,0,1,1,1,"((r7:1.21434e+06,(r5:1.0..."
998,998,0,1,1,1,"((r7:2.05576e+06,(r0:1.2..."


In [9]:
model.seqs

array([[3, 1, 3, ..., 0, 3, 3],
       [2, 1, 1, ..., 0, 2, 2],
       [2, 1, 3, ..., 1, 2, 2],
       ...,
       [0, 3, 3, ..., 1, 3, 3],
       [2, 1, 1, ..., 0, 2, 3],
       [2, 1, 3, ..., 1, 3, 2]], dtype=uint8)

#### 3b) Simulate a bunch of loci

In [10]:
# simulate N loci of len L 
model.sim_loci(100, 3000)

In [11]:
# view the genealogies and stats in a table
model.df

,locus,start,end,nbps,nsnps,genealogy
0,0,0,21,21,1,"((r7:1.56192e+06,(r3:1.1..."
1,0,21,121,100,12,"((r7:1.56192e+06,(r3:1.1..."
2,0,121,166,45,9,"((r7:1.56192e+06,(r3:1.1..."
3,0,166,191,25,5,"((r7:1.56192e+06,(r3:1.1..."
4,0,191,203,12,1,"((r7:1.56192e+06,(r3:1.1..."
...,...,...,...,...,...,...
3842,99,2727,2809,82,23,"((r3:2.18634e+06,(r0:945..."
3843,99,2809,2885,76,17,"((r3:2.18634e+06,(r0:945..."
3844,99,2885,2927,42,12,"((r3:2.18634e+06,(r0:945..."
3845,99,2927,2971,44,12,"((r3:992058,(r0:945210,r..."


In [12]:
# view sequence data as an array
model.seqs

array([[[2, 1, 0, ..., 2, 2, 3],
        [2, 1, 0, ..., 2, 1, 3],
        [2, 1, 0, ..., 2, 1, 3],
        ...,
        [2, 1, 0, ..., 2, 2, 3],
        [2, 1, 0, ..., 2, 1, 3],
        [2, 1, 0, ..., 2, 1, 3]],

       [[2, 3, 3, ..., 0, 1, 2],
        [2, 3, 3, ..., 0, 1, 2],
        [2, 3, 3, ..., 0, 1, 2],
        ...,
        [2, 3, 3, ..., 0, 1, 2],
        [2, 3, 3, ..., 0, 1, 2],
        [2, 3, 3, ..., 0, 1, 2]],

       [[2, 1, 0, ..., 2, 3, 1],
        [2, 1, 0, ..., 2, 3, 1],
        [2, 1, 0, ..., 2, 0, 1],
        ...,
        [2, 1, 0, ..., 2, 0, 1],
        [2, 1, 0, ..., 2, 3, 1],
        [2, 1, 0, ..., 2, 0, 1]],

       ...,

       [[3, 0, 0, ..., 3, 2, 1],
        [3, 0, 0, ..., 3, 2, 1],
        [3, 0, 0, ..., 3, 2, 1],
        ...,
        [3, 0, 0, ..., 3, 2, 1],
        [3, 0, 0, ..., 3, 2, 1],
        [3, 0, 0, ..., 3, 2, 1]],

       [[2, 2, 2, ..., 0, 1, 1],
        [2, 2, 2, ..., 0, 1, 1],
        [2, 2, 2, ..., 0, 1, 1],
        ...,
        [2, 2, 2, ..., 

#### 4) Infer a gene tree for each locus

In [13]:
# infer a tree for every locus
model.infer_gene_trees(inference_method='raxml')

In [14]:
# inferred_tree column is now added to the dataframe!
model.df

,locus,start,end,nbps,nsnps,genealogy,inferred_tree
0,0,0,21,21,1,"((r7:1.56192e+06,(r3:1.1...",((r4:0.02105516863155426...
1,0,21,121,100,12,"((r7:1.56192e+06,(r3:1.1...",((r4:0.02105516863155426...
2,0,121,166,45,9,"((r7:1.56192e+06,(r3:1.1...",((r4:0.02105516863155426...
3,0,166,191,25,5,"((r7:1.56192e+06,(r3:1.1...",((r4:0.02105516863155426...
4,0,191,203,12,1,"((r7:1.56192e+06,(r3:1.1...",((r4:0.02105516863155426...
...,...,...,...,...,...,...,...
3842,99,2727,2809,82,23,"((r3:2.18634e+06,(r0:945...",((r3:0.01625070763953974...
3843,99,2809,2885,76,17,"((r3:2.18634e+06,(r0:945...",((r3:0.01625070763953974...
3844,99,2885,2927,42,12,"((r3:2.18634e+06,(r0:945...",((r3:0.01625070763953974...
3845,99,2927,2971,44,12,"((r3:992058,(r0:945210,r...",((r3:0.01625070763953974...
